---
## Task 2 -Data Pre-processing
### Berfin Duman 
April 3, 2023

---

**Task 2:** Find a data set which is suitable for regression analysis and consists of a mix of numerical, nominal, and ordinal variables. Look for the cases, where at least one of the variable has missing values. If not, you can randomly delete a very small portion of one of the variables.
Design a machine learning pipeline where you scale the numerical features and encode the nominal and ordinal features along with imputing the missing values.

#### Instructions to follow:

- Include all your codes here. Be sure that your code is CLEAN, READABLE, and REPRODUCIBLE.
- Put your data set into a **datasets** folder.
- Put your images (if available) into an **images** folder.
- Please return a NICE and CLEAR homework. Otherwise, it will not be graded.
- Please write YOUR OWN code. **DO NOT copy** my codes or someone else's codes.

In [65]:
import sklearn
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression


In [37]:
from sklearn import set_config
set_config(transform_output="pandas") 
#When we fit our dataset with a process and then transform it, 
#this code should be executable in order not to lose the column numbers in the dataframe.

First of all, I searched a lot of datasets. I know about the titanic dataset, which is kind of very popular, but I've never done a large scale analysis like this before and actually this course is my first lesson in data, I think this start could be good for me. Honestly, although I've browsed the uci and kaggle website a lot, I haven't been able to find a dataset that is a mix of numeric, nominal and ordinal variables. I hope it won't be a problem.

Titanic dataset columns includes these feature: 
1. PassengerID: Identify number of passenger. 
2. Survived: After diseaster, this variable indicate whether the passenger survived. (0: dead, 1: survived)
3. Pclass: Variable indicating the class in which the passenger is traveling.  (1 = First class, 2 = Second class, 3 = Third class)
4. Name: Name of passenger. 
5. Sex: Gender of passanger (male, female)
6. Age: Age of passenger.

7. SibSp: Number of siblings/spouses traveling with the passenger
8. Parch: Number of parents/children traveling with the passenger
9. Ticket: Ticket number
10. Fare: Price of ticket
11. Cabin: Passenger's cabin number
12. Embarked: Passenger embarkation port (C = Cherbourg, Q = Queenstown, S = Southampton)


In [38]:
data=pd.read_csv("../datasets/train.csv",index_col="PassengerId") #i choose PassengerId as index column. because this feature include unique number
data.shape

(891, 11)

In [39]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


Before I start, I will nominalize the data qualified as 1 2 3 in PClass to First, Second, Third to better observe OrdinalEncoding.

In [40]:
map_for_pclass = {1: 'First', 2: 'Second', 3: 'Third'}
data['Pclass'] = data['Pclass'].replace(map_for_pclass)

In [41]:
def dataset_analyze(data):
    for column in data.columns:
        print(column)
        print("value counts: ", len(data[column].value_counts()), "value types:", data[column].dtype, "non-null:", data[column].notnull().sum())
        print("**********")

In [42]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,Third,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,First,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,Third,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,First,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,Third,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,Second,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,First,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,Third,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [43]:
dataset_analyze(data)
#

Survived
value counts:  2 value types: int64 non-null: 891
**********
Pclass
value counts:  3 value types: object non-null: 891
**********
Name
value counts:  891 value types: object non-null: 891
**********
Sex
value counts:  2 value types: object non-null: 891
**********
Age
value counts:  88 value types: float64 non-null: 714
**********
SibSp
value counts:  7 value types: int64 non-null: 891
**********
Parch
value counts:  7 value types: int64 non-null: 891
**********
Ticket
value counts:  681 value types: object non-null: 891
**********
Fare
value counts:  248 value types: float64 non-null: 891
**********
Cabin
value counts:  147 value types: object non-null: 204
**********
Embarked
value counts:  3 value types: object non-null: 889
**********


Here is our dataset features and value_counts:
+  Survived  : 2 , int64 -> Survived is target variable
+ Pclass  : 3 , int64 -> this Pclass includes ordinal variables, I encoding this column with OrdinalEncoder class with sklearn. 
+ Name  : 891 , object -> I deleted this column because this and PassengerId column are same contribution for dataset. And I pick PassengerID, which I choose ad index_col and i drop Name column, because this for this analyzing Name feature is not necessary.
+ Sex  : 2 , object -> I encoding Sex feature with OneHotEncoding.
+ Age  : 88 , float64  
+ SibSp  : 7 , int64 
+ Parch  : 7 , int64
+ Fare  : 248 , float64 \
Age,SibSp, Parch and Fare features includes numeric variables, so I don't touch these features and put the model directly for analyzing.
+ Embarked  : 3 , object -> This feature included 3 different as C,Q and S option. I encoding using OneHotEncoding libraries too.
+ Ticket  : 681 , object
+ Cabin  : 147 , object \
The above 2 features actually confused me;I think, ticket and cabin are categorical variable and i should encoding this features. But i searched many works with Titanic dataset, almost everyone dropped this two feature. But I went a little deeper, and i realized some special analyzes about this two columns.


-About Ticket - 

    End of analysis which , i referred this work in references part, about focus on ticket, the result that the data should be discarded and this idea suited me because people, which has same ticketIds, can be relative and this situation seems other columns as SibSp and Parch; of course there will be different situations but still this I thought it was not necessary to add 680 new columns by doing OneHotEncoding with column. I thought that I could delete this data as my remaining data set would be suitable for making OneHotEncoding, OrdinalEncoding methods that I need to analyze as homework. Target variable has already numerical so i don't need  encoding target variable using LabelEncoding.

-About Cabin -

    Actually i dont want directly drop Cabin column. This information which in Cabin column learned by searching on Chatgpt. And answer is: 
    "The cabin letters in the Titanic dataset represent the deck where the passenger's cabin was located. The letters are a combination of the deck level and the side of the ship where the cabin was located. For example, the letter "C" represents the third deck, and cabins on the starboard side (right side) of the ship were labeled with odd numbers, while those on the port side (left side) were labeled with even numbers."
    And i decide this column parse a letter and numbers; and this letter that given information about deck level so hold with me and drop following numbers. But when i run data.Cabin.value_counts() for take information about Cabin values; i realized some different format; some of rows included many cabins for example <B57 B59 B63 B66> and <C23 C25 C27> and worse some of rows included different deck cabins like <F E69> <F G63>. And for these situations, i decided hold first cabin numbers and parse it for taking deck level. 
    
    Maybe it is not necessary this columns so i added result with drop cabin columns, overall I don't think it's necessary for homework anyway.. But this part a kind of crazy part for me, lets start.

 

In [44]:
data = data.drop(["Name", "Ticket"], axis=1)

In [45]:
def set_deck(cabin_numb):
    deck= str(cabin_numb).split()[0][0]
    room = str(cabin_numb).split()[0][1:]
    if deck=="n" or  room== "": 
        deck=np.nan
        room=np.nan
    else:
        room=int(room)
    return deck, room
data[["Deck", "Room"]] = data["Cabin"].apply(lambda x: set_deck(x)).apply(pd.Series)

In [46]:
data.Room.value_counts() 


33.0     7
6.0      6
23.0     5
22.0     5
2.0      5
        ..
21.0     1
12.0     1
63.0     1
14.0     1
148.0    1
Name: Room, Length: 92, dtype: int64

In [47]:
data.Deck.value_counts()

C    59
B    47
E    32
D    30
A    15
F     9
G     4
Name: Deck, dtype: int64

As I said above, I will discard the room line. We will only deal with the deck

In [48]:
data = data.drop("Room", axis=1)
data = data.drop("Cabin", axis=1)

In [49]:
dataset_analyze(data)

Survived
value counts:  2 value types: int64 non-null: 891
**********
Pclass
value counts:  3 value types: object non-null: 891
**********
Sex
value counts:  2 value types: object non-null: 891
**********
Age
value counts:  88 value types: float64 non-null: 714
**********
SibSp
value counts:  7 value types: int64 non-null: 891
**********
Parch
value counts:  7 value types: int64 non-null: 891
**********
Fare
value counts:  248 value types: float64 non-null: 891
**********
Embarked
value counts:  3 value types: object non-null: 889
**********
Deck
value counts:  7 value types: object non-null: 196
**********


## Adding Null Variable
There are null values in Age and Desk, as an extra, I will add null values to Sex, Fare and PClass so that nominal, numeric and ordinal variables all have null values. 

In [50]:
def add_null_values(data, columns, percent):
    data = data.copy()
    for col in columns:
        num_nulls = int(data.shape[0] * percent) 
        #the number of nulls based on the number of rows of the data and the percentage given#
        indices = random.sample(list(data.index), num_nulls)
        #generates as many random values as nulls in the number of rows range
        data.loc[indices, col] = np.nan
        #sets the value in the specified row and column to null
    return data

In [51]:
data=add_null_values(data, ["Sex","Fare", "Pclass"], 0.05)


In [52]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
PassengerId,,,,,,,,,
1,0,Third,male,22.0,1,0,7.2500,S,NaN
2,1,First,female,38.0,1,0,71.2833,C,C
3,1,Third,female,26.0,0,0,7.9250,S,NaN
4,1,First,female,35.0,1,0,53.1000,S,C
5,0,Third,male,35.0,0,0,8.0500,S,NaN
...,...,...,...,...,...,...,...,...,...
887,0,Second,male,27.0,0,0,13.0000,S,NaN
888,1,First,female,19.0,0,0,30.0000,S,B
889,0,Third,female,NaN,1,2,23.4500,S,NaN


In [53]:
dataset_analyze(data)

Survived
value counts:  2 value types: int64 non-null: 891
**********
Pclass
value counts:  3 value types: object non-null: 847
**********
Sex
value counts:  2 value types: object non-null: 847
**********
Age
value counts:  88 value types: float64 non-null: 714
**********
SibSp
value counts:  7 value types: int64 non-null: 891
**********
Parch
value counts:  7 value types: int64 non-null: 891
**********
Fare
value counts:  245 value types: float64 non-null: 847
**********
Embarked
value counts:  3 value types: object non-null: 889
**********
Deck
value counts:  7 value types: object non-null: 196
**********


In [54]:
new_data= add_null_values(data,["Sex","Pclass"] , 0.05) #as you said above -a very small portion- set 0.05 percent 
##approximately 44 variable set as null

## Imputation in missing values 
Age, Desk, Sex and PClass features has missing values:  \
For "Age" features using -> SimpleImputer(strategy='mean')\
For "Desk, Sex, PClass" features using -> SimpleImputer(strategy="most_frequent")

## Scaling Operations 
For numerical features, we can apply scaling operator like StandartScalar()

## Encoding Operations

Survived-> Set as y variable and if this column is

Age -> Already numerical \
SibSp -> Already numerical \
Parch -> Already numerical \
Fare-> Already numerical 

Pclass -> Ordinal Encoding (1,2,3 ) \
Sex -> (male,female) OneHotEncoding(if_binary=True) \
Embarked -> (S, C, Q) OneHotEncoding() \
Deck ->(C, B, E, D, A, F, G) OneHotEncoding() \

pClass = ["First", "Second", "Third"] #0,1,2,3
SimpleImputer(strategy="most_frequent")
OrdinalEncoder(categories=pClass) 
OneHotEncode(categories=["Sex", "Embarked", "Deck"], drop= 'first')

In [55]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
PassengerId,,,,,,,,
15,Third,female,14.0,0,0,NaN,S,NaN
420,Third,female,10.0,0,2,24.1500,S,NaN
750,Third,male,31.0,0,0,7.7500,Q,NaN
267,Third,male,16.0,4,1,39.6875,S,NaN
710,Third,male,NaN,1,1,15.2458,C,NaN
...,...,...,...,...,...,...,...,...
835,NaN,male,18.0,0,0,8.3000,S,NaN
514,First,female,54.0,1,0,59.4000,C,NaN
770,Third,male,32.0,0,0,8.3625,S,NaN


In [56]:
y=data.Survived
X_train, X_test, y_train, y_test = train_test_split(data.drop("Survived", axis=1), y, test_size=0.3, random_state=1300)

## Pipeline Design

- Design your machine learning pipeline here.

In [66]:
# Define the column transformer
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(drop='first',sparse=False)
)

ordinal_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder(categories=[["First", "Second", "Third"]])
)

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, ['Age', 'SibSp', 'Parch', 'Fare']),
        ('ordinal', ordinal_transformer, ['Pclass']),
        ('categorical', categorical_transformer, ['Sex', 'Embarked', 'Deck'])
    ])

# Define the pipeline
pipe = make_pipeline(preprocessor, LogisticRegression())

pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['First',
                                                                                               'Second',
                                                                                               'Third']]))]),
                                                  ['Pclass']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['Sex', 'Embarked',
                                                   'Deck'])])),
                ('logisticregression', LogisticRegression())])

## Pipeline Results

- Put your regression analysis (which is based on a pipeline object) results here. Comment on the results.

In [67]:
# Fit the pipeline to the training data with deck column
pipe.fit(X_train, y_train)
pipe.score(X_test,y_test)

C:\Users\Asus\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.832089552238806

### Drop Deck Feature on Dataset

In [68]:
X_train_nodeck= X_train.drop("Deck",axis=1)
X_test_nodec=X_test.drop("Deck",axis=1)

In [69]:
# Define the column transformer
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(drop='first',sparse=False)
)

ordinal_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder(categories=[["First", "Second", "Third"]])
)

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, ['Age', 'SibSp', 'Parch', 'Fare']),
        ('ordinal', ordinal_transformer, ['Pclass']),
        ('categorical', categorical_transformer, ['Sex', 'Embarked'])
    ])

# Define the pipeline
pipe = make_pipeline(preprocessor, LogisticRegression())

pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['First',
                                                                                               'Second',
                                                                                               'Third']]))]),
                                                  ['Pclass']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [73]:
# Fit the pipeline to the training data
pipe.fit(X_train_nodeck, y_train)


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['First',
                                                                                               'Second',
                                                                                               'Third']]))]),
                                                  ['Pclass']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [75]:
pipe.score(X_test_nodec, y_test) #without Deck column

0.8059701492537313

By doing a little experimental work here, I broke the cabin feature and divided it into two as deck and room class. I looked at the results in two different cases when I added the deck to the model and dropped the deck because I thought the Deck class had an importance in the target variable. The fact that Deck is in the model seems to contribute to the positive model:
R2 score with desk column:
    0.832089552238806
R2 score without desk column:
    0.8059701492537313

In [76]:
import session_info
session_info.show()